In [1]:
import pandas as pd
import os
import glob

In [2]:
df = pd.read_csv('image_url.csv')

In [3]:
df_itemcat = pd.read_csv('item_cat_mapping.csv')

In [4]:
df1 = pd.merge(df,df_itemcat, how = 'inner', on = ['item_name','item_name'])

In [5]:
df1.head()

,row_number,item_cat_x,item_imgurl,item_code,item_name,url,url_num,item_cat_y
0,0,Lassi,https://res.cloudinary.com/swiggy/image/upload...,xapptmddiry8qhfaozbu,Sweet Lassi,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,Lassi
1,128,"Lassi, Juice & Milkshake Festival",https://res.cloudinary.com/swiggy/image/upload...,jxxg0eik96ruzeacwdhw,Sweet Lassi,https://www.swiggy.com/bangalore/vasudev-adiga...,3,Lassi
2,172,Lassi & Juice Festival,https://res.cloudinary.com/swiggy/image/upload...,u0pivupljynaa8riooxc,Sweet Lassi,https://www.swiggy.com/bangalore/vasudev-adiga...,4,Lassi
3,870,Beverages,https://res.cloudinary.com/swiggy/image/upload...,o42mufaaofvebna95he5,Sweet Lassi,https://www.swiggy.com/bangalore/desi-v-desi-c...,43,Lassi
4,885,Beverages,https://res.cloudinary.com/swiggy/image/upload...,ghmcdaipu3ywfv8c4mkr,Sweet Lassi,https://www.swiggy.com/bangalore/desi-vdesi-sh...,44,Lassi


In [24]:
df_labels = df1[['item_code','item_cat_y']]

In [26]:
df_labels.columns = ['item_code','item_cat']

In [27]:
df_labels.to_csv('labels.csv')

In [6]:
files_list = glob.glob("/home/paperspace/Nutrition_Journal/Images/*.jpg")
df_filenames = pd.DataFrame(files_list)
df_filenames.columns = ['file_name']

df_filenames['item_code'] = df_filenames['file_name'].apply(lambda x: x.split('/')[-1])
df_filenames['item_code'] = df_filenames['item_code'].apply(lambda x: x.split('.')[-2])

In [7]:
df1 = pd.merge(df, df_filenames, how='left', on=['item_code', 'item_code'])

In [8]:
df1.head()

,row_number,item_cat,item_imgurl,item_code,item_name,url,url_num,file_name
0,0,Lassi,https://res.cloudinary.com/swiggy/image/upload...,xapptmddiry8qhfaozbu,Sweet Lassi,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,/home/paperspace/Nutrition_Journal/Images/xapp...
1,1,Lassi,https://res.cloudinary.com/swiggy/image/upload...,drssb2tezmlldscjcxji,Fruit Lassi,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,/home/paperspace/Nutrition_Journal/Images/drss...
2,2,Lassi,https://res.cloudinary.com/swiggy/image/upload...,vtw9ywlvmm0qrybwod5m,Strawberry Lassi,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,/home/paperspace/Nutrition_Journal/Images/vtw9...
3,3,Lassi,https://res.cloudinary.com/swiggy/image/upload...,suubbt273xx6wgwlaavp,Dry Fruit Lassi,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,/home/paperspace/Nutrition_Journal/Images/suub...
4,4,Ice Cream,https://res.cloudinary.com/swiggy/image/upload...,d5gqx5mvoc0pcwnty8rd,Fruit Salad with Ice cream,https://www.swiggy.com/bangalore/lassi-shop-ce...,0,/home/paperspace/Nutrition_Journal/Images/d5gq...


In [11]:
df2 = df1.groupby(['item_cat','item_name'])['item_name'].size().reset_index(name ='size')

In [12]:
df2.head()

,item_cat,item_name,size
0,Pizzas,All Time Favourite Margherita Pizza,5
1,Pizzas,Khoune Andhra Chicken Pizza,2
2,Pizzas,Khoune Grilled Chicken Pizza,2
3,Pizzas,Khoune BBQ Paneer Pizza,2
4,Pizzas,Khoune Chicken Tikka Pizza,2


In [13]:
len(df2)

51836

In [14]:
df2.to_csv('food_items.csv')

In [15]:
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk import bigrams 
import string

In [16]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt','RT','via']

In [17]:
def preprocess(word):
    tokens = nltk.word_tokenize(word)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in stop and token.isalpha()]
    return tokens

In [20]:
df2['tokens'] = df2.apply(lambda row: preprocess(row['item_name']), axis=1)

In [21]:
df2.head(10)

,item_cat,item_name,size,tokens
0,Pizzas,All Time Favourite Margherita Pizza,5,"[time, favourite, margherita, pizza]"
1,Pizzas,Khoune Andhra Chicken Pizza,2,"[khoune, andhra, chicken, pizza]"
2,Pizzas,Khoune Grilled Chicken Pizza,2,"[khoune, grilled, chicken, pizza]"
3,Pizzas,Khoune BBQ Paneer Pizza,2,"[khoune, bbq, paneer, pizza]"
4,Pizzas,Khoune Chicken Tikka Pizza,2,"[khoune, chicken, tikka, pizza]"
5,Pizzas,Khoune Fried Mushroom Pizza,2,"[khoune, fried, mushroom, pizza]"
6,Pizzas,Khoune Grilled Paneer Pizza,2,"[khoune, grilled, paneer, pizza]"
7,Pizzas,Khoune Spicy Veg Pizza,2,"[khoune, spicy, veg, pizza]"
8,111 Varieties Dosa,American Chop suey Dosa,2,"[american, chop, suey, dosa]"
9,111 Varieties Dosa,Delight Dosa,2,"[delight, dosa]"


In [22]:
a = df2['item_name'].str.cat(sep=' ')
words = preprocess(a)
word_dist = nltk.FreqDist(words)
rslt = pd.DataFrame(word_dist.most_common(10000),columns=['Word', 'Frequency'])

In [23]:
rslt[rslt.Frequency > 100].to_csv('food_cat.csv')

In [63]:
df[df.item_name.str.contains('Chocolate')]

,row_number,item_cat,item_imgurl,item_code,item_name,url,url_num
5,5,Ice Cream,https://res.cloudinary.com/swiggy/image/upload...,csykytvwidwufqku1p0w,Death By Chocolate,https://www.swiggy.com/bangalore/lassi-shop-ce...,0
44,44,Riviting Desserts,https://res.cloudinary.com/swiggy/image/upload...,g5nuj6o5q0dbqg81mzdu,B & W Chocolate Cake (Eggless),https://www.swiggy.com/bangalore/baba-da-dhaba...,1
92,92,"Lassi, Juice & Milkshake Festival",https://res.cloudinary.com/swiggy/image/upload...,a4ey3rmb6ice3d2tpsoy,Chocolate Milkshake,https://www.swiggy.com/bangalore/vasudev-adiga...,2
296,296,Desserts & Cookies,https://res.cloudinary.com/swiggy/image/upload...,tj9yxt73yjrss2qlgwqo,Dark Chocolate Brownies,https://www.swiggy.com/bangalore/fresh-presser...,10
417,417,Freshly Baked,https://res.cloudinary.com/swiggy/image/upload...,v3enwhz1eb6zutfteqrc,Chocolate Gems Cupcake (Pack of 2),https://www.swiggy.com/bangalore/the-brekkie-s...,14
437,437,Sundaes,https://res.cloudinary.com/swiggy/image/upload...,esq6ie7ybhxqwqbsh2oz,Hot Chocolate Fudge,https://www.swiggy.com/bangalore/art-of-deligh...,16
439,439,Sundaes,https://res.cloudinary.com/swiggy/image/upload...,v2uchi6rh5twipsfz8ao,Chocolate Hysteria Sundae,https://www.swiggy.com/bangalore/art-of-deligh...,16
440,440,Sundaes,https://res.cloudinary.com/swiggy/image/upload...,t5j4hrbxxh7hxqg2fb9v,Chocolate Superior Sundae,https://www.swiggy.com/bangalore/art-of-deligh...,16
451,451,Ice Creams,https://res.cloudinary.com/swiggy/image/upload...,tfckkzeqmtdbdxs92gq1,Belgian Chocolate Ice cream,https://www.swiggy.com/bangalore/art-of-deligh...,16
453,453,Ice Creams,https://res.cloudinary.com/swiggy/image/upload...,hpfdnhxlgxgssj9iwgyi,White Chocolate Nutella Ice cream,https://www.swiggy.com/bangalore/art-of-deligh...,16


### Image file names

In [105]:
files_list = glob.glob("/home/paperspace/Nutrition_Journal/Images/*.jpg")
df_filenames = pd.DataFrame(files_list)

df_filenames.columns = ['file_name']

df_filenames['item_code'] = df_filenames['file_name'].apply(lambda x: x.split('/')[-1])
df_filenames['item_code'] = df_filenames['item_code'].apply(lambda x: x.split('.')[-2])

In [106]:
df_filenames.head(10)

,file_name,item_code
0,/home/paperspace/Nutrition_Journal/Images/p5wr...,p5wruk8uxbktempggpit
1,/home/paperspace/Nutrition_Journal/Images/h1d1...,h1d1qxm0h0b4fgkshv0x
2,/home/paperspace/Nutrition_Journal/Images/swde...,swdexuemz7txryirzwdi
3,/home/paperspace/Nutrition_Journal/Images/sbht...,sbhtbzexqj6ljrote0pv
4,/home/paperspace/Nutrition_Journal/Images/k0yx...,k0yx8syph46kveurs1pe
5,/home/paperspace/Nutrition_Journal/Images/srey...,sreyilguv8xqrl9gj2tb
6,/home/paperspace/Nutrition_Journal/Images/cc3j...,cc3jwv5aqq8vrtlfbp8n
7,/home/paperspace/Nutrition_Journal/Images/qmjc...,qmjcxpgypjypn7pjulm0
8,/home/paperspace/Nutrition_Journal/Images/uqzo...,uqzovptasbocuaoaf89a
9,/home/paperspace/Nutrition_Journal/Images/wo2l...,wo2l4wlko4prdjx0g1gs


In [93]:
% pwd

'/home/paperspace/Nutrition_Journal'

In [107]:
df_labels = pd.read_csv('food_resnet/labels.csv')

In [108]:
df_labels.head()

,item_code,item_cat
0,xapptmddiry8qhfaozbu,Lassi
1,jxxg0eik96ruzeacwdhw,Lassi
2,u0pivupljynaa8riooxc,Lassi
3,o42mufaaofvebna95he5,Lassi
4,ghmcdaipu3ywfv8c4mkr,Lassi


In [109]:
df_map = pd.merge(df_labels, df_filenames, how = 'inner', on = ['item_code','item_code'])

In [110]:
print (len(df_labels))
print (len(df_filenames))
print (len(df_map))

115294
115287
115294


In [111]:
df_map.head(10)

,item_code,item_cat,file_name
0,xapptmddiry8qhfaozbu,Lassi,/home/paperspace/Nutrition_Journal/Images/xapp...
1,jxxg0eik96ruzeacwdhw,Lassi,/home/paperspace/Nutrition_Journal/Images/jxxg...
2,u0pivupljynaa8riooxc,Lassi,/home/paperspace/Nutrition_Journal/Images/u0pi...
3,u0pivupljynaa8riooxc,Juice,/home/paperspace/Nutrition_Journal/Images/u0pi...
4,o42mufaaofvebna95he5,Lassi,/home/paperspace/Nutrition_Journal/Images/o42m...
5,ghmcdaipu3ywfv8c4mkr,Lassi,/home/paperspace/Nutrition_Journal/Images/ghmc...
6,cas2iowwn6xnkq4z4xhb,Lassi,/home/paperspace/Nutrition_Journal/Images/cas2...
7,q1gj8dfmrjsdjr6sh3uv,Lassi,/home/paperspace/Nutrition_Journal/Images/q1gj...
8,nbh9bv7h9kp0r99trc8y,Lassi,/home/paperspace/Nutrition_Journal/Images/nbh9...
9,sga7da6lkvyxmg9fyvmy,Lassi,/home/paperspace/Nutrition_Journal/Images/sga7...


In [99]:
df_map0 = pd.merge(df_filenames,df_labels, how = 'left', on = ['item_code','item_code'])

In [100]:
delfilenames = df_map0[df_map0['item_cat'].isnull()].file_name

In [101]:
#delfilenames.to_csv('delfilenames.csv')

In [48]:
## csv to txt

In [102]:
df_map0[df_map0['item_cat'].isnull()]

,file_name,item_code,item_cat
25,/home/paperspace/Nutrition_Journal/Images/vt2h...,vt2hswqgrns9tlttabtj,NaN
37,/home/paperspace/Nutrition_Journal/Images/ajwa...,ajwagio6ajyhg28ki5fd,NaN
47,/home/paperspace/Nutrition_Journal/Images/sdy7...,sdy7d7xxkhw301pnbwsc,NaN
57,/home/paperspace/Nutrition_Journal/Images/t5ku...,t5kumkqdfqlwhgtcfrfg,NaN
76,/home/paperspace/Nutrition_Journal/Images/x0ew...,x0ewbx9zgzrgdfdrh0ub,NaN
87,/home/paperspace/Nutrition_Journal/Images/vxkg...,vxkg7vyjmw9fwdgivls7,NaN
133,/home/paperspace/Nutrition_Journal/Images/qxkr...,qxkrdesgot1jiggwxrye,NaN
149,/home/paperspace/Nutrition_Journal/Images/uzir...,uzirpn3oc2k6e4v1dolr,NaN
169,/home/paperspace/Nutrition_Journal/Images/bdvk...,bdvkk6b6dzuzmwszryf0,NaN
226,/home/paperspace/Nutrition_Journal/Images/luak...,luakwnwihtkcooyfeb9r,NaN


In [103]:
filelist = list(delfilenames)

In [104]:
for file in filelist:
    os.remove(file)

In [112]:
import glob
import shutil
import os

src_dir = "/home/paperspace/Nutrition_Journal/Images"
dst_dir = "/home/paperspace/Nutrition_Journal/food_resnet/train"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)

In [113]:
os.chdir('/home/paperspace/Nutrition_Journal/food_resnet/train')